In [ ]:
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import h5py

from holodeck import plot, detstats
from holodeck.constants import YR, MSOL, MPC
import holodeck as holo

# Read in data

In [ ]:
lib_path = '/Users/emigardiner/GWs/holodeck/output/awg_output/uniform-07a_new_n500_r100_f40'
hdf_file = h5py.File(lib_path+'/sam_lib.hdf5')
print(hdf_file.keys())
params = hdf_file['sample_params'][...]
hc_ss = hdf_file['hc_ss'][...]
hc_bg = hdf_file['hc_bg'][...]
sspar = hdf_file['sspar'][...]
bgpar = hdf_file['bgpar'][...]
fobs = hdf_file['fobs'][:]
hdf_file.close()

In [ ]:
space_class = holo.param_spaces.PS_Uniform_07A
space = space_class(holo.log, 0, None, None)
print(space.param_names)
print(params[0])
print(params[-1])
print(params.shape)

In [ ]:
ds_file = np.load(lib_path+'/detstats/psrs50_sigma8.28e-06/detstats.npz')
df_ss = ds_file['df_ss']
df_bg = ds_file['df_bg']
dp_ss = ds_file['dp_ss']
dp_bg = ds_file['dp_bg']
ds_file.close()

# Best Samples

In [ ]:
nsort = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=detstats.HC_REF15_10YR,  fidx=1)
print(nsort[0:10])

# Binary Parameters

### just using best sample

In [ ]:
print(sspar.shape)

In [ ]:
freqs = np.repeat(fobs, hc_ss[0,0,:,:].size).reshape(hc_ss[0].shape)
print(freqs.shape)
print(freqs[:,0,0]*YR)

## Plot: hc vs bin pars

In [ ]:
nn = nsort[0]
plot_path = '/Users/emigardiner/GWs/holodeck/ecg-notebooks/parameter_investigation/plots_07a'


def draw_ss_bin_pars(axs, hc_ss, sspar, fobs, labels, colors):

    x1 = sspar[0,...] / MSOL
    x2 = sspar[1,...]
    x3 = holo.cosmo.comoving_distance(sspar[2,...]) 
    x4 = np.repeat(fobs, hc_ss[0,:,:].size).reshape(hc_ss.shape) *YR
    yy = hc_ss

    for ii, xx in enumerate([x1, x2, x3, x4]):
        axs[ii].set_xlabel(labels[ii])
        for rr in range(len(yy[0])):
            axs[ii].scatter(xx[:,rr,:].flatten(), yy[:,rr,:].flatten(), marker='o', s=15, alpha=0.1, color=colors[rr])

def draw_bg_bin_pars(axs, hc_bg, bgpar, fobs, color='k'):
    x1 = bgpar[0,...] / MSOL 
    x2 = bgpar[1,...]
    x3 = holo.cosmo.comoving_distance(bgpar[2,...]) 
    x4 = np.repeat(fobs, hc_bg[0,:].size).reshape(hc_bg.shape) *YR
    yy = hc_bg

    for ii, xx in enumerate([x1, x2, x3, x4]):
        # axs[ii].set_xlabel(labels[ii])
        for rr in range(len(yy[0])):
            axs[ii].scatter(xx[:,rr], yy[:,rr], marker='x', s=15, alpha=0.1, color=color)

def plot_bin_pars(nn, hc_ss, hc_bg, sspar, bgpar, fobs):
    fig, axs = plot.figax(nrows=2, ncols=2, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN, sharey=True,
                        figsize=(10,7))

    axs = axs.reshape(axs.size)
    labels=['Total Mass, $M$ $(M_\odot)$', 'Mass Ratio, $q$', 'Initial Comoving Distance, $d_c$ (Mpc)', plot.LABEL_GW_FREQUENCY_YR]
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0,0])))
    
    draw_bg_bin_pars(axs, hc_bg[nn], bgpar[nn], fobs)
    draw_ss_bin_pars(axs, hc_ss[nn], sspar[nn], fobs, labels, colors)
    fig.tight_layout()
    return fig

fig = plot_bin_pars(nsort[0], hc_ss, hc_bg, sspar, bgpar, fobs)

In [ ]:
print('%04d' % nn)

In [ ]:
nn = nsort[0]


def draw_ss_bin_pars_fast(axs, hc_ss, sspar, fobs, labels, color='r'):

    x1 = sspar[0,...] / MSOL
    x2 = sspar[1,...]
    x3 = holo.cosmo.comoving_distance(sspar[2,...]) 
    x4 = np.repeat(fobs, hc_ss[0,:,:].size).reshape(hc_ss.shape) *YR
    yy = hc_ss

    for ii, xx in enumerate([x1, x2, x3, x4]):
        axs[ii].set_xlabel(labels[ii])
        axs[ii].scatter(xx.flatten(), yy.flatten(), marker='o', s=15, alpha=0.1, color=color)

def draw_bg_bin_pars_fast(axs, hc_bg, bgpar, fobs, color='k'):
    x1 = bgpar[0,...] / MSOL 
    x2 = bgpar[1,...]
    x3 = holo.cosmo.comoving_distance(bgpar[2,...]) 
    x4 = np.repeat(fobs, hc_bg[0,:].size).reshape(hc_bg.shape) *YR
    yy = hc_bg

    for ii, xx in enumerate([x1, x2, x3, x4]):
        axs[ii].scatter(xx.flatten(), yy.flatten(), marker='x', s=15, alpha=0.1, color=color)

def draw_sample_text(fig, nn, params, param_names, 
                     xx=0.1, yy=-0.025):
    text = ''
    for pp, name in enumerate(param_names):
        text = text+"'%s'=%.2e, " % (name, params[nn,pp])
        if pp == int(len(param_names)/2):
            text = text+'\n'
    fig.text(xx, yy, text, fontsize=10, color='k', alpha=0.75)
    

def plot_bin_pars_fast(nn, hc_ss, hc_bg, sspar, bgpar, fobs, param_names=space.param_names):
    fig, axs = plot.figax(nrows=2, ncols=2, ylabel='$h_c$', sharey=True,
                        figsize=(9,6))

    axs = axs.reshape(axs.size)
    labels=['$M$ $(M_\odot)$',  '$q$', ' $d_c$ (Mpc)', '$f\ (\mathrm{yr}^{-1})$']
    
    draw_bg_bin_pars_fast(axs, hc_bg[nn], bgpar[nn], fobs)
    draw_ss_bin_pars_fast(axs, hc_ss[nn], sspar[nn], fobs, labels)

    draw_sample_text(fig, nn, params, param_names, xx=0.1, yy=-0.025)
    fig.tight_layout()
    return fig

for nn in nsort[:5]:
    fig = plot_bin_pars_fast(nn, hc_ss, hc_bg, sspar, bgpar, fobs)

What about for the worst models? Or the rest?

In [ ]:
for ii in range(len(nsort)):
    nn = nsort[ii]
    print('ii=%d, nn=%d' % (ii,nn))
    fig = plot_bin_pars_fast(nn, hc_ss, hc_bg, sspar, bgpar, fobs)
    fig.savefig(plot_path+'/hc_vs_bins_rank%04d_p%04d.png' % (ii, nn), dpi=100)
    plt.close(fig)

## Plot: SNR vs bin pars

In [ ]:
nn = nsort[0]


def draw_ss_bin_pars(axs, hc_ss, sspar, fobs, labels, colors):

    x1 = sspar[0,...] / MSOL
    x2 = sspar[1,...]
    x3 = holo.cosmo.comoving_distance(sspar[2,...]) 
    x4 = np.repeat(fobs, hc_ss[0,:,:].size).reshape(hc_ss.shape) *YR
    yy = hc_ss

    for ii, xx in enumerate([x1, x2, x3, x4]):
        axs[ii].set_xlabel(labels[ii])
        for rr in range(len(yy[0])):
            axs[ii].scatter(xx[:,rr,:].flatten(), yy[:,rr,:].flatten(), marker='o', s=15, alpha=0.1, color=colors[rr])

def draw_bg_bin_pars(axs, hc_bg, bgpar, fobs, color='k'):
    x1 = bgpar[0,...] / MSOL
    x2 = bgpar[1,...]
    x3 = holo.cosmo.comoving_distance(bgpar[2,...]) 
    x4 = np.repeat(fobs, hc_bg[0,:].size).reshape(hc_bg.shape) *YR
    yy = hc_bg

    for ii, xx in enumerate([x1, x2, x3, x4]):
        # axs[ii].set_xlabel(labels[ii])
        for rr in range(len(yy[0])):
            axs[ii].scatter(xx[:,rr], yy[:,rr], marker='x', s=15, alpha=0.1, color=color)

def plot_bin_pars(nn, hc_ss, hc_bg, sspar, bgpar, fobs):
    fig, axs = plot.figax(nrows=2, ncols=2, ylabel=plot.LABEL_CHARACTERISTIC_STRAIN, sharey=True,
                        figsize=(10,7))

    axs = axs.reshape(axs.size)
    labels=['Mass, $M$ $(M_\odot)$', 'Mass Ratio, $q$', 'Initial Comoving Distance, $d_c$ (Mpc)', plot.LABEL_GW_FREQUENCY_YR]
    colors = cm.rainbow(np.linspace(0,1,len(hc_ss[0,0])))
    
    draw_bg_bin_pars(axs, hc_bg[nn], bgpar[nn], fobs)
    draw_ss_bin_pars(axs, hc_ss[nn], sspar[nn], fobs, labels, colors)
    fig.tight_layout()
    return fig

fig = plot_bin_pars(nsort[0], hc_ss, hc_bg, sspar, bgpar, fobs)

# DetFrac vs. Parameter

## hard_time
I expect with shorter hardening times, we would see more sources (loud and bg).

In [ ]:
def draw_df(ax, xx, df_ss=None, df_bg=None, label_ss='SS', label_bg='BG'):
    if df_bg is not None:
        ax.scatter(xx, df_bg, color='cornflowerblue', label=label_bg, 
                   marker='d', alpha=0.5)
    if df_ss is not None:
        ax.scatter(xx, df_ss, color='orangered', label=label_ss, 
                   marker='o', alpha=0.5)
        
fig, ax = plot.figax(xlabel="'hard_time'", ylabel='Detection Fraction')
draw_df(ax, params[:,0], df_ss, df_bg)

In [ ]:
print(holo.utils.stats(params[:,1]))

sample astro pars

In [ ]:
fig, axs = plot.figax(nrows=2, ncols=3, figsize=(12, 7), xscale='linear', yscale='linear')
for aa, ax in enumerate(fig.axes[:5]):
    ax.set_ylabel('%s' % space.param_names[aa]) 
    ax.scatter(np.arange(len(params)), params[:,aa])
axs[1,0].set_xlabel('Sample')
axs[1,1].set_xlabel('Sample')

fig.tight_layout()

## Plot: DF vs astro pars

In [ ]:
fig, axs = plot.figax(nrows=2, ncols=3, figsize=(12, 7), xscale='linear', sharey=True)
for aa, ax in enumerate(fig.axes[:5]):
    draw_df(ax, xx=params[:,aa], df_ss=df_ss, df_bg=df_bg)
    ax.set_xlabel('%s' % space.param_names[aa]) 
    ax.set_ylim(10**-3, 10**0.1)
axs[0,0].set_ylabel('Detection Fraction')
axs[1,0].set_ylabel('Detection Fraction')

fig.tight_layout()
# draw_df(ax, params[:,0], df_ss, df_bg)

# Ranking best

In [ ]:
nsort, fidx, hc_tt, hc_ref = detstats.rank_samples(hc_ss, hc_bg, fobs, hc_ref=detstats.HC_REF15_10YR,  fidx=1, ret_all=True)

hc_tt = np.sqrt(np.sum(hc_ss**2, axis=-1) + hc_bg**2) # (N,F,R)
hc_dif = np.abs(hc_tt[:,fidx,:] - hc_ref) # (N,R)
hc_med_of_dif = np.median(hc_dif, axis=-1) # (N,) # median realization difference from ref freqs

In [ ]:
hc_dif_of_med = np.median(hc_tt[:,fidx,:], axis=-1) # (N,)
hc_dif_of_med = np.abs(hc_dif_of_med - hc_ref)

compare this ranking vs. the nsort ranking (which just used median hc_tt across all reals), out of curiosity

In [ ]:
fig, ax = plot.figax(xlabel='sample ranking by nsort', ylabel='$|h_c - h_{c,\mathrm{ref}}|$', 
                     xscale='linear', figsize=(10,5)
                     )
ax.plot(np.arange(len(nsort)), hc_med_of_dif[nsort], '-o', label='median of differences',
           alpha=0.5, ms=3)
ax.plot(np.arange(len(nsort)), hc_dif_of_med[nsort], '-o', label='differences of median',
           alpha=0.5, ms=3)
ax.legend()

## Plot: Ref-Diff vs Astro Pars

In [ ]:
print(hc_med_of_dif.shape, df_bg.shape)

In [ ]:
for pp in range(len(space.param_names)):
    fig, ax = plot.figax(
        xlabel='$|h_{c,\mathrm{ref}} - h_c|$, decreasing likelihood', 
        ylabel=space.param_names[pp],
        yscale='linear')
    ax.plot(hc_dif_of_med[nsort], params[:,pp][nsort], '-o', alpha=0.5)

In [ ]:
def draw_1dim(ax, xx, yy_ss=None, label_ss=None, color_ss='orangered',
              yy_bg=None, label_bg=None, color_bg='cornflowerblue'):
    if yy_bg is not None:
        ax.scatter(xx, yy_bg, color=color_bg, label=label_bg, 
                   marker='d', alpha=0.5)
    if yy_ss is not None:
        print(xx.shape, yy_ss.shape)
        ax.scatter(xx, yy_ss, color=color_ss, label=label_ss, 
                   marker='o', alpha=0.5)

def draw_diff_vs_rank(ax):
    # plot dif vs rank
    ax.plot(np.arange(len(nsort)), hc_med_of_dif[nsort], '-o', label='median of differences',
            alpha=0.5, ms=3)
    ax.plot(np.arange(len(nsort)), hc_dif_of_med[nsort], '-o', label='differences of median',
            alpha=0.5, ms=3)
    ax.set_xscale('linear')
    ax.legend()


fig, axs = plot.figax(nrows=2, ncols=3, figsize=(12, 7), sharey=True, xscale='linear')
for aa, ax in enumerate(fig.axes[:5]):
    draw_1dim(ax, xx=params[:,aa], 
              yy_ss=hc_med_of_dif, label_ss='median of diffs', color_ss='tab:blue',
              yy_bg = hc_dif_of_med, label_bg='diff of medians', color_bg='tab:orange') # smaller dif = more likely
    ax.set_xlabel('%s' % space.param_names[aa]) 
    # ax.set_ylim(10**-3, 10**0.1)
draw_diff_vs_rank(axs[-1,-1])



axs[0,0].set_ylabel('$|h_{c\mathrm{ref}} - h_c|$ (decreasing likelihood)')
axs[1,0].set_ylabel('$|h_{c\mathrm{ref}} - h_c|$ (decreasing likelihood)')

fig.tight_layout()
# draw_df(ax, params[:,0], df_ss, df_bg)